# 1 libraries

In [2]:
import numpy as np
import pandas as pd
import os
import re
import networkx as nx
#import scona as scn
import pickle
from numpy import genfromtxt

# 2 Load data

In [3]:
count_data_dir = './ADNI_structural_graph_count_threshold_20'
count_targ_folder = os.listdir(count_data_dir)
count_graph_list=list(count_targ_folder)
count_graph_list.sort()
len(count_graph_list)

179

In [4]:
with open('./ADNI_structural_graph_count_threshold_20/ADNI_structural_graph_count_threshold_20_4081.pickle', 'rb') as f:
    test = pickle.load(f)

In [5]:
test

{'labels': 1,
 'num_nodes': 84,
 'num_edges': 2540,
 'edge_index': array([[ 2.,  2.,  3., ..., 83., 83., 83.],
        [ 0.,  1.,  0., ..., 77., 78., 82.]]),
 'edge_feat': array([1486.,  821.,   30., ...,  146.,  171.,   93.]),
 'node_feat': array([['2.548', '16362'],
        ['2.154', '11230'],
        ['nan', '3251.7'],
        ['2.864', '9626'],
        ['2.928', '5414'],
        ['2.935', '1734'],
        ['2.488', '7781'],
        ['3.98', '1883'],
        ['2.727', '7434'],
        ['2.477', '8118'],
        ['2.644', '9488'],
        ['2.58', '1774'],
        ['2.433', '11992'],
        ['2.916', '9079'],
        ['2.463', '1623'],
        ['2.909', '9729'],
        ['nan', '39863.2'],
        ['2.412', '4451'],
        ['2.246', '1605'],
        ['nan', '503.0'],
        ['nan', '1363.2'],
        ['nan', '3112.7'],
        ['2.719', '9161'],
        ['3.093', '2094'],
        ['2.772', '9998'],
        ['nan', '1285.6'],
        ['2.493', '4240'],
        ['1.727', '1913'],
  

# 4 count

### 4-1 graph level - graph label, num_edge_list, num_node_list

In [11]:
graph_label = pd.DataFrame(columns=['diagnosis'])
num_edge = pd.DataFrame(columns=['num_edge'])
num_node = pd.DataFrame(columns=['num_node'])

In [7]:
type(test['num_nodes'])

int

In [12]:
#!rm -rf abcd-struct-count
#!mkdir ./ADNI-struct-count
for i in range(len(count_graph_list)):
    data = os.path.join(count_data_dir, count_graph_list[i])
    subject_id = count_graph_list[i].split('_')[4].split('.')[0]
    with open(data, 'rb') as f:
        graph_data = pickle.load(f)
        
    # 1 graph label
    diagnosis = graph_data['labels'].astype(np.float16)
    graph_label = graph_label.append(pd.DataFrame([diagnosis], columns=['diagnosis']),
                                     ignore_index=True)
    
    # 2 num_edge_list
    num_edge_score = int(graph_data['num_edges'])
    num_edge = num_edge.append(pd.DataFrame([num_edge_score], columns=['num_edge']),
                               ignore_index=True)
    
    
    # 3 num_node_list
    num_node_score = int(graph_data['num_nodes'])
    num_node = num_node.append(pd.DataFrame([num_node_score], columns=['num_node']),
                               ignore_index=True)
    

In [9]:
num_edge

,num_edge
0,2546
1,2426
2,2440
3,2540
4,2485
...,...
174,2403
175,2465
176,2423
177,2545


In [13]:
graph_label

,diagnosis
0,1.0
1,3.0
2,1.0
3,1.0
4,3.0
...,...
174,1.0
175,1.0
176,1.0
177,1.0


In [14]:
num_node

,num_node
0,84
1,84
2,84
3,84
4,84
...,...
174,84
175,84
176,84
177,84


In [15]:
# save files    
graph_label.to_csv('./ADNI-struct-count/graph-label.csv.gz', index=False, header=None)
num_edge.to_csv('./ADNI-struct-count/num-edge-list.csv.gz', index=False, header=None)
num_node.to_csv('./ADNI-struct-count/num-node-list.csv.gz', index=False, header=None)

### 4-2 edge level - edge_feat, edge

In [24]:
edge_feat_list = []
edge_index_source_list=[]
edge_index_target_list=[]

In [25]:
test['edge_index'][0][0]

2.0

In [26]:
test['edge_feat'][0]

1486.0

In [27]:
type(test['edge_index'][0][0])

numpy.float64

In [28]:
for i in range(len(count_graph_list)):
    data = os.path.join(count_data_dir, count_graph_list[i])
    subject_id = count_graph_list[i].split('_')[-1].split('.')[0]
    with open(data, 'rb') as f:
        graph_data = pickle.load(f)
    
    for j in range(graph_data['num_edges']):
        # 4 edge_feat
        if graph_data['edge_feat'][j] == 1.0:
            edge_feat_list.append(1)
        else:
            edge_feat_list.append(int(1*graph_data['edge_feat'][j].astype(np.float64)))
        ### fa는 10, count는 1, 나머진 다 10000 곱하기
              
        # 5 edge_index
        edge_index_source_list.append(round(graph_data['edge_index'][0][j]))
        edge_index_target_list.append(round(graph_data['edge_index'][1][j]))

In [29]:
edge_feat=pd.DataFrame({'struct_conn': edge_feat_list})
edge_index=pd.DataFrame({'source': edge_index_source_list, 'target' : edge_index_target_list})

In [30]:
edge_feat

,struct_conn
0,343
1,72
2,23
3,313
4,45
...,...
428521,230
428522,402
428523,42
428524,32


In [31]:
edge_index

,source,target
0,2,0
1,2,1
2,4,0
3,5,0
4,5,2
...,...,...
428521,83,77
428522,83,78
428523,83,80
428524,83,81


In [32]:
# save edge feat
edge_feat.to_csv('./ADNI-struct-count/edge-feat.csv.gz', index=False, header=None)

# save edge index
edge_index.to_csv('./ADNI-struct-count/edge.csv.gz', index=False, header=None)

### 4-3 node level - node feat

In [33]:
node_feat_cor_thick_list=[]
node_feat_vol_list=[]

In [34]:
type(round(graph_data['node_feat'].T[1][3].astype(np.float64)))

int

In [38]:
for i in range(len(count_graph_list)):
    data = os.path.join(count_data_dir, count_graph_list[i])
    subject_id = count_graph_list[i].split('_')[-1].split('.')[0]
    with open(data, 'rb') as f:
        graph_data = pickle.load(f)
    
    for j in range(graph_data['num_nodes']):
        # 6 node_feat
        ## 6-1 cortical thickness
        if graph_data['node_feat'].T[0][j] == 'nan':
            node_feat_cor_thick_list.append(-1)
        else:
            node_feat_cor_thick_list.append(int(10 * graph_data['node_feat'].T[0][j].astype(np.float32)))
        
        ## 6-2 volume - scaled
        if graph_data['node_feat'].T[1][j] == 'nan':
            node_feat_vol_list.append(-1)
        else:
            node_feat_vol_list.append(int(0.001 * graph_data['node_feat'].T[1][j].astype(np.float32)))

In [39]:
node_feat=pd.DataFrame({'cortical_thickness': node_feat_cor_thick_list, 'volume' : node_feat_vol_list})

In [40]:
node_feat # integer

,cortical_thickness,volume
0,25,18
1,19,9
2,-1,3
3,26,8
4,29,5
...,...,...
15031,24,2
15032,21,3
15033,24,1
15034,-1,41


In [42]:
# save node feat
node_feat.to_csv('./ADNI-struct-count/node-feat.csv.gz', index=False, header=None)